# 1 - Introduction

In [1]:
import json

In [2]:
import pandas as pd
import numpy as np

In [3]:
def num_rows(df):
    print('Dataframe has {} observations'.format(df.shape[0]))

In [4]:
OUT_FILE = "../hackathon_template/energy_flows.json"
IN_DIR = "../data/sankey/"

This notebook is used to generate the data that is fed into the Sankey diagram. It aims to be as transparent as possible in the manipulations that are performed on the data.

# Read in main data

The "raw" data is stored in multiple _csv_ files in the /data/sankey directory. Start by reading in the main sheet, "CAN15_Mapped", provided by the organizers of the hackathon.

In [5]:
df = pd.read_csv(IN_DIR + "CAN15_Mapped.csv", dtype={"Value":'float64'})
num_rows(df)
df.head()

Dataframe has 85 observations


,Ref_Date,GEO,FUEL,SUPPLY,Vector,Coordinate,Value,Input,Output
0,2015,Canada,Total refined petroleum products,Inter-product transfers,v54272424,1.14.6,-386892.0,Refined petroleum products,Inter-product transfers
1,2015,Canada,Gas plant natural gas liquids (NGL's),Other adjustments,v54272889,1.6.7,-124573.0,NGLs,Other adjustments
2,2015,Canada,Natural gas,Inter-product transfers,v54272864,1.5.6,-112297.0,Natural gas,Inter-product transfers
3,2015,Canada,Total refined petroleum products,Stock variation,v54272423,1.14.5,-35833.0,Stock variation,Refined petroleum products
4,2015,Canada,Gas plant natural gas liquids (NGL's),Stock variation,v54272888,1.6.5,-17653.0,Stock variation,NGLs


# Cleaning

Some cleaning on the provided file is necessary for various reasons.

## Detailed Primary Production

In this section, we will incorporate details on the production of primary electricity. We start by reading in this data from the Excel file.

In [6]:
detailed_primary = pd.read_csv(IN_DIR + "1270007mapped.csv")
detailed_primary.head()

,Ref_Date,GEO,FUEL,SUPPLY,Vector,Coordinate,Value,Input,Output
0,2015,Canada,Hydraulic turbine (3),Production,v0000,1.1.1,1362628,Hydro,"Primary electricity, hydro and nuclear"
1,2015,Canada,Tidal power turbine (4),Production,v0001,1.1.2,46,Tidal,"Primary electricity, hydro and nuclear"
2,2015,Canada,Wind power turbine (5),Production,v0002,1.1.3,96108,Wind,"Primary electricity, hydro and nuclear"
3,2015,Canada,Solar (10),Production,v0003,1.1.4,5136,Solar,"Primary electricity, hydro and nuclear"
4,2015,Canada,Other types of electricity generation,Production,v0004,1.1.5,502,Other Electricity,"Primary electricity, hydro and nuclear"


Now drop the link between general production and primary electricity.

In [7]:
df = df[~((df["Input"] == 'Production') & (df["Output"] == 'Primary electricity, hydro and nuclear'))]
num_rows(df)

Dataframe has 84 observations


Now append on the primary production details.

In [8]:
df = pd.concat([df, detailed_primary])
num_rows(df)

Dataframe has 90 observations


## Imputations

A great deal of flows in the main sheet are suppressed due to confidentiality concerns. Currently, these observations have all been imputed as having values of _33_. For some flows, this is far too low. Improved imputation can be found in the _Imputations_ tab and will replace the current values.

In [9]:
imputations = pd.read_csv(IN_DIR+"Imputations.csv", dtype={"Value":np.float})
num_rows(imputations)

Dataframe has 10 observations


Again, we will drop the original imputations and then append the improved imputed observations.

In [10]:
df = df[df["Value"] != 33]

In [11]:
df = pd.concat([df, imputations])
num_rows(df)

Dataframe has 90 observations


# Negatives

Some of the values are currently negative. They need to be changed to positive while creating new names to differentiate between energy supply and uses.

## Stock variations

Positive stock variations become "Stock deposits" and the input and output columns will be switched.

In [12]:
slicer = (df['Input'] == 'Stock variation') & (df['Value'] > 0)
df[slicer].head()

,Ref_Date,GEO,FUEL,SUPPLY,Vector,Coordinate,Value,Input,Output
23,2015,Canada,Crude oil,Stock variation,v54272828,1.4.5,10178.0,Stock variation,Crude oil
25,2015,Canada,Total coal,Stock variation,v54272744,1.29.5,10504.0,Stock variation,Coal
48,2015,Canada,Natural gas,Stock variation,v54272863,1.5.5,95673.0,Stock variation,Natural gas


In [13]:
df.loc[slicer, "Input"] = df.loc[slicer, "Output"]
df.loc[slicer, "Output"] = 'Stock deposits'
df[slicer].head()

,Ref_Date,GEO,FUEL,SUPPLY,Vector,Coordinate,Value,Input,Output
23,2015,Canada,Crude oil,Stock variation,v54272828,1.4.5,10178.0,Crude oil,Stock deposits
25,2015,Canada,Total coal,Stock variation,v54272744,1.29.5,10504.0,Coal,Stock deposits
48,2015,Canada,Natural gas,Stock variation,v54272863,1.5.5,95673.0,Natural gas,Stock deposits


Negative stock variations become "Stock withdrawals", but there is no need to switch the input and output columns.

In [14]:
slicer = (df['Input'] == 'Stock variation') & (df['Value'] < 0)
df[slicer].head()

,Ref_Date,GEO,FUEL,SUPPLY,Vector,Coordinate,Value,Input,Output
3,2015,Canada,Total refined petroleum products,Stock variation,v54272423,1.14.5,-35833.0,Stock variation,Refined petroleum products
4,2015,Canada,Gas plant natural gas liquids (NGL's),Stock variation,v54272888,1.6.5,-17653.0,Stock variation,NGLs
5,2015,Canada,Coke,Stock variation,v54272370,1.10.5,-2321.0,Stock variation,Coal products


In [15]:
df.loc[slicer, "Input"] = 'Stock withdrawals'
df.loc[slicer, "Value"] = -df.loc[slicer, "Value"]
df[slicer].head()

,Ref_Date,GEO,FUEL,SUPPLY,Vector,Coordinate,Value,Input,Output
3,2015,Canada,Total refined petroleum products,Stock variation,v54272423,1.14.5,35833.0,Stock withdrawals,Refined petroleum products
4,2015,Canada,Gas plant natural gas liquids (NGL's),Stock variation,v54272888,1.6.5,17653.0,Stock withdrawals,NGLs
5,2015,Canada,Coke,Stock variation,v54272370,1.10.5,2321.0,Stock withdrawals,Coal products


## Other adjustments

Positive 'Other adjustments' are switched to the 'Input' column and renamed to 'Other supplies'. Following instructions from Footnote 7 from CANSIM 128-0016.

In [16]:
slicer = (df['Output'] == 'Other adjustments') & (df['Value'] >= 0)
df[slicer].head()

,Ref_Date,GEO,FUEL,SUPPLY,Vector,Coordinate,Value,Input,Output
33,2015,Canada,Total coal,Other adjustments,v54325467,1.29.7,33469.0,Coal,Other adjustments
51,2015,Canada,Natural gas,Other adjustments,v54272865,1.5.7,187930.0,Natural gas,Other adjustments
72,2015,Canada,Crude oil,Other adjustments,v65584138,1.4.7,908378.0,Crude oil,Other adjustments


In [17]:
df.loc[slicer, "Output"] = df.loc[slicer, "Input"]
df.loc[slicer, "Input"] = 'Other supplies'
df.loc[slicer].head()

,Ref_Date,GEO,FUEL,SUPPLY,Vector,Coordinate,Value,Input,Output
33,2015,Canada,Total coal,Other adjustments,v54325467,1.29.7,33469.0,Other supplies,Coal
51,2015,Canada,Natural gas,Other adjustments,v54272865,1.5.7,187930.0,Other supplies,Natural gas
72,2015,Canada,Crude oil,Other adjustments,v65584138,1.4.7,908378.0,Other supplies,Crude oil


Negative 'Other adjustments' become 'Other losses' and the negative value becomes positive

In [18]:
slicer = (df['Output'] == 'Other adjustments')& (df['Value'] < 0)
df[slicer].head()

,Ref_Date,GEO,FUEL,SUPPLY,Vector,Coordinate,Value,Input,Output
1,2015,Canada,Gas plant natural gas liquids (NGL's),Other adjustments,v54272889,1.6.7,-124573.0,NGLs,Other adjustments


In [19]:
df.loc[slicer, "Output"] = 'Other losses'
df.loc[slicer, "Value"] = -df.loc[slicer, "Value"]
df.loc[slicer].head()

,Ref_Date,GEO,FUEL,SUPPLY,Vector,Coordinate,Value,Input,Output
1,2015,Canada,Gas plant natural gas liquids (NGL's),Other adjustments,v54272889,1.6.7,124573.0,NGLs,Other losses


## Interproduct transfers

Per footnote 5 in Table 128-0016, _Inter-product transfers_ "Shows the transfer of similar products between different product forms; for example, still gas to natural gas, natural gas to produce hydrogen in petroleum refineries." Observations with negative values will have the "Input" and "Output" columns switched and have the positive values made positive. Bote that there are no 'Inter-product transfers' links with positive values, for this particular dataset.

In [20]:
slicer = (df["Output"] == 'Inter-product transfers') & (df['Value'] > 0)
df[slicer]

,Ref_Date,GEO,FUEL,SUPPLY,Vector,Coordinate,Value,Input,Output


Now make necessary changes to links with negative values.

In [21]:
slicer = (df["Output"] == 'Inter-product transfers') & (df['Value'] < 0)
df[slicer].head()

,Ref_Date,GEO,FUEL,SUPPLY,Vector,Coordinate,Value,Input,Output
0,2015,Canada,Total refined petroleum products,Inter-product transfers,v54272424,1.14.6,-386892.0,Refined petroleum products,Inter-product transfers
2,2015,Canada,Natural gas,Inter-product transfers,v54272864,1.5.6,-112297.0,Natural gas,Inter-product transfers


In [22]:
df.loc[slicer, "Value"] = -df.loc[slicer, "Value"]
df.loc[slicer, "Output"] = df.loc[slicer, 'Input']
df.loc[slicer, "Input"] = 'Inter-product transfers'
df.loc[slicer].head()

,Ref_Date,GEO,FUEL,SUPPLY,Vector,Coordinate,Value,Input,Output
0,2015,Canada,Total refined petroleum products,Inter-product transfers,v54272424,1.14.6,386892.0,Inter-product transfers,Refined petroleum products
2,2015,Canada,Natural gas,Inter-product transfers,v54272864,1.5.6,112297.0,Inter-product transfers,Natural gas


## Rename

Rename some of the nodes to have slightly shorter names as well as to group some smaller steps in the supply chain. This will make things easier to display.

In [23]:
df["Output"].unique()

array(['Refined petroleum products', 'Other losses', 'Natural gas',
       'NGLs', 'Coal products', 'Statistical difference',
       'Producer consumption', 'Secondary electricity, thermal',
       'Residential', 'Non-energy use', 'Agriculture', 'Stock deposits',
       'Primary electricity, hydro and nuclear', 'Total transportation',
       'Total industrial', 'Public administration', 'Coal',
       'Commercial and other institutional', 'Exports',
       'Electricity generation', 'Crude oil'], dtype=object)

In [24]:
mapper_dict = {"Primary electricity, hydro and nuclear":"Primary electricity",
               "Refined petroleum products":"Refined petroleum",
               "Secondary electricity, thermal":"Secondary electricity",
               "Public administration":"Other services",
               "Commercial and other institutional":"Other services",
               "Residential":"Other services"}
def mapper(src):
    return mapper_dict.get(src, src)

df["Input"] = df["Input"].map(mapper)
df["Output"] = df["Output"].map(mapper)

## Final Cleaning

The final cleaning involves dropping observations with 0 values, and also Statistical Difference since it is not very informative.

In [25]:
df = df[df["Output"] != "Statistical difference"]

In [26]:
df = df[df["Value"] > 0]

In [27]:
#sum to make certain unique links
df = df[["Input", "Output", "Value"]].groupby(["Input", "Output"], as_index=False).sum()

# Output JSONs

The final step is to output the Pandas data frame into to two JSON files that can be read by the D3 visualization. Start by creating unique nodes.

In [28]:
nodes = pd.concat([df["Input"], df["Output"]]).unique()
node_dict = {node:i for i, node in enumerate(nodes)}

In [29]:
df["source"] = df["Input"].map(node_dict).astype(int)
df["target"] = df["Output"].map(node_dict).astype(int)
df.head()

,Input,Output,Value,source,target
0,Coal,Coal products,87307.0,0,1
1,Coal,Exports,712731.0,0,20
2,Coal,Non-energy use,2383.0,0,21
3,Coal,Other services,305.0,0,22
4,Coal,Producer consumption,11.0,0,23


In [30]:
#make value column lower case
df=df.rename(columns={"Value":"value"})

In [31]:
#make nodes into a list of dicts
nodes_json = [dict(name=node) for node in nodes]
links_json = df[["source", "target", "value"]].to_dict(orient="records")

In [32]:
#json.dump doesn't like numpy dtypes
for link in links_json:
    link["source"] = int(link["source"])
    link["target"] = int(link["target"])
    link["value"] = int(link["value"])

In [33]:
json_output = {"links":links_json,
    "nodes":nodes_json,
              }

In [34]:
#now output
with open(OUT_FILE, 'w') as fp:
    json.dump(json_output, fp, indent=0)